In [1]:
#Libraries

#Data Analysis Libs
import pandas as pd
import numpy as np

In [3]:
od_matrix = pd.read_csv('/local/tarciso/masters/data/bus_trips/latest/otp-od/2017_06_16_aa_od_trips.csv', parse_dates=['start_time','end_time'])

In [4]:
od_matrix.head()

,cardNum,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,start_time,from_stop_lat,from_stop_lon,to_stop_id,end_time,to_stop_lat,to_stop_lon,leg_duration
0,844324.0,4700,1,1,342,BA120,10.0,34915.0,2017-06-16 14:50:47,-25.396340,-49.202170,30448.0,2017-06-16 15:09:47,-25.393374,-49.240917,0 days 00:19:00.000000000
1,948116.0,6671,2,1,684,HA610,1.0,39372.0,2017-06-16 04:48:26,-25.593450,-49.333880,31053.0,2017-06-16 05:12:47,-25.513098,-49.295303,0 days 00:24:21.000000000
2,948116.0,6671,2,3,204,HL309,1.0,26252.0,2017-06-16 05:29:56,-25.512983,-49.294505,25917.0,2017-06-16 05:57:00,-25.426995,-49.264729,0 days 00:27:04.000000000
3,948116.0,6671,2,5,361,BC906,1.0,29889.0,2017-06-16 06:21:06,-25.426910,-49.263080,28979.0,2017-06-16 06:27:10,-25.417287,-49.249938,0 days 00:06:04.000000000
4,948116.0,6672,5,1,216,BA603,7.0,28979.0,2017-06-16 15:25:37,-25.417287,-49.249938,28332.0,2017-06-16 16:00:52,-25.476127,-49.292362,0 days 00:35:15.000000000


In [25]:
len(od_matrix)

188

In [40]:
boardings_df = od_matrix.filter(['route','busCode','tripNum','from_stop_id','start_time']) \
                        .groupby(['route','busCode','tripNum','from_stop_id']) \
                        .count() \
                        .reset_index() \
                        .rename(index=str, columns={'from_stop_id':'stop_id', 'start_time':'boarding_cnt'}) \
                        .sort_values('boarding_cnt', ascending=False)

In [41]:
boardings_df.head()

,route,busCode,tripNum,stop_id,boarding_cnt
37,204,HL309,1.0,26252.0,8
105,603,HD245,1.0,26296.0,8
94,602,GD330,1.0,27121.0,4
133,650,HA285,1.0,39734.0,3
131,650,HA285,1.0,36293.0,3


In [43]:
alightings_df = od_matrix.filter(['route','busCode','tripNum','to_stop_id','end_time']) \
                        .groupby(['route','busCode','tripNum','to_stop_id']) \
                        .count() \
                        .reset_index() \
                        .rename(index=str, columns={'to_stop_id':'stop_id', 'end_time':'alighting_cnt'}) \
                        .sort_values('alighting_cnt', ascending=False)

In [44]:
alightings_df.head()

,route,busCode,tripNum,stop_id,alighting_cnt
139,650,HA285,1.0,31050.0,8
147,684,HA610,1.0,31053.0,5
90,548,GA209,1.0,4661.0,4
35,204,HL309,1.0,8727.0,3
96,602,GD330,1.0,26263.0,2


In [49]:
passenger_flows = boardings_df.merge(alightings_df, how='outer') \
                                .sort_values(['boarding_cnt','alighting_cnt'])

In [50]:
passenger_flows.head(50)

,route,busCode,tripNum,stop_id,boarding_cnt,alighting_cnt
84,203,BD145,5.0,26199.0,1.0,1.0
29,548,GA209,1.0,4661.0,1.0,4.0
10,607,HL318,4.0,25900.0,1.0,NaN
11,603,HE710,6.0,25400.0,1.0,NaN
12,607,HL314,4.0,26245.0,1.0,NaN
13,607,HL314,5.0,25900.0,1.0,NaN
14,20,DR103,1.0,26228.0,1.0,NaN
15,607,HL319,4.0,25900.0,1.0,NaN
16,617,HA284,11.0,31051.0,1.0,NaN
17,619,HA002,1.0,36280.0,1.0,NaN
